# QL Decomposition using Householder Matrices    

In [69]:
import numpy as np
import pandas as pd

In [70]:
#Create a random test vector and random test matrix
testVec = np.random.rand(10)
testMat = np.random.rand(10,3)

### Building the Householder Vector

First, we create a function to compute the Householder vector $v$ for a given vector $x$

In [71]:
def housesame(x):
    m = len(x)
    v = x.copy().astype(np.float)
    ess_normSq = np.linalg.norm(x[0:m-1],ord = 2)**2 #The 2-norm squared of the essential part
    
    if ess_normSq == 0: #If x is already zero-ed out
        beta = 0 #The Householder update should be the identity
    else:
        x_norm = np.linalg.norm(x,ord = 2) #The 2-norm of the input x
        
        if x[m-1] <= 0:
            v_update = x[m-1] - x_norm
        else:
            v_update = x[m-1] + x_norm        
        
        v[m-1] = v_update
        beta = 2/(np.linalg.norm(v,ord = 2)**2)
        
    return [v,beta]

In [72]:
v,beta = housesame(testVec)
update = (np.identity(len(testVec)) - beta*np.outer(v,v))
np.dot(update,testVec)

array([  4.16333634e-17,   6.93889390e-17,   5.55111512e-17,
         1.11022302e-16,   2.77555756e-17,   1.94289029e-16,
         0.00000000e+00,   5.55111512e-17,   5.55111512e-17,
        -1.59639475e+00])

Now, another function to create the oppositve Householder vector

In [73]:
def houseopposite(x):
    m = len(x)
    v = x.copy().astype(np.float)
    ess_normSq = np.linalg.norm(x[0:m-1],ord = 2)**2 #The 2-norm squared of the essential part
    
    if ess_normSq == 0: #If x is already zero-ed out
        beta = 0 #The Householder update should be the identity
    else:
        x_norm = np.linalg.norm(x,ord = 2) #The 2-norm of the input x
        
        if x[m-1] <= 0:
            v_update = x[m-1] + x_norm
        else:
            v_update = x[m-1] - x_norm        
        
        v[m-1] = v_update
        beta = 2/(np.linalg.norm(v,ord = 2)**2)
        
    return [v,beta]

In [74]:
#Check the output
v,beta = houseopposite(testVec)
update = (np.identity(len(testVec)) - beta*np.outer(v,v))
np.dot(update,testVec)

array([  0.00000000e+00,  -1.11022302e-16,   0.00000000e+00,
        -5.55111512e-17,  -4.16333634e-17,  -5.55111512e-17,
        -1.38777878e-17,  -5.55111512e-17,   5.55111512e-17,
         1.59639475e+00])

In [75]:
# All elements except the last one are close to numpy machine precision 2.2204460492503131e-16

### Applying the Householder Matrices

In [78]:
def householdersame(A,precision = 10):
    rows,cols = np.shape(A)
    q = np.identity(rows)
    
    for k in range(cols,0,-1):
        [v,beta] = housesame(A[0:(rows-(cols-k)),k-1])
        p = np.identity(len(v)) - beta*np.outer(v,v)
        update = np.identity(rows)
        update[0:np.shape(p)[0],0:np.shape(p)[1]] = p
        A = np.matmul(update,A) 
    
    #Store Q
    q = np.matmul(update,q)

    
    #Now, permute the rows of A to get the lower triangular part on the top
    Ap = A.copy()
    Ap[0:cols,:]  = A[rows - cols:rows,:]
    Ap[rows - cols:rows,:]  = A[0:cols,:]
        
    return [q.round(precision),Ap.round(precision)]

In [79]:
print testMat.round(4)
print "~~~~~~~"
[q,lower] = householdersame(testMat,precision = 2)
print q
print lower

[[ 0.2688  0.5115  0.9645]
 [ 0.2495  0.0274  0.9875]
 [ 0.4353  0.1027  0.0219]
 [ 0.5178  0.3178  0.0602]
 [ 0.8767  0.4434  0.9587]
 [ 0.8628  0.7484  0.5724]
 [ 0.667   0.1453  0.3884]
 [ 0.788   0.1539  0.9882]
 [ 0.8844  0.1161  0.903 ]
 [ 0.5003  0.4942  0.5335]]
~~~~~~~
[[ 0.76 -0.18  0.19  0.18  0.07  0.12  0.18  0.53  0.    0.  ]
 [-0.18  0.87  0.14  0.13  0.05  0.09  0.13  0.38  0.    0.  ]
 [ 0.19  0.14  0.86 -0.14 -0.05 -0.09 -0.14 -0.4   0.    0.  ]
 [ 0.18  0.13 -0.14  0.87 -0.05 -0.09 -0.14 -0.39  0.    0.  ]
 [ 0.07  0.05 -0.05 -0.05  0.98 -0.03 -0.05 -0.14  0.    0.  ]
 [ 0.12  0.09 -0.09 -0.09 -0.03  0.94 -0.09 -0.26  0.    0.  ]
 [ 0.18  0.13 -0.14 -0.14 -0.05 -0.09  0.86 -0.4   0.    0.  ]
 [ 0.53  0.38 -0.4  -0.39 -0.14 -0.26 -0.4  -0.14  0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    1.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    1.  ]]
[[-0.95  0.   -0.  ]
 [ 0.59  0.84 -0.  ]
 [-1.72 -0.85 -2.32]
 [-0.   -0.    0.  ]
 [ 0. 

In [80]:
#Look at just the lower triag matrix
print lower

[[-0.95  0.   -0.  ]
 [ 0.59  0.84 -0.  ]
 [-1.72 -0.85 -2.32]
 [-0.   -0.    0.  ]
 [ 0.    0.    0.  ]
 [-0.   -0.    0.  ]
 [-0.    0.   -0.  ]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]
 [-0.   -0.   -0.  ]]


In [81]:
def householderopposite(A,precision = 10):
    rows,cols = np.shape(A)
    q = np.identity(rows)

    for k in range(cols,0,-1):
        [v,beta] = houseopposite(A[0:(rows-(cols-k)),k-1])
        p = np.identity(len(v)) - beta*np.outer(v,v)
        update = np.identity(rows)
        update[0:np.shape(p)[0],0:np.shape(p)[1]] = p
        A = np.matmul(update,A) 
    
    #Store Q
    q = np.matmul(update,q)

    
    #Now, permute the rows of A to get the lower triangular part on the top
    Ap = A.copy()
    Ap[0:cols,:]  = A[rows - cols:rows,:]
    Ap[rows - cols:rows,:]  = A[0:cols,:]
        
    return [q.round(precision),Ap.round(precision)]

In [82]:
print "Test Matrix:"
print testMat.round(4)

[q,lower] = householderopposite(testMat,precision = 4)
print "Q:"
print q
print "L:"
print lower

Test Matrix:
[[ 0.2688  0.5115  0.9645]
 [ 0.2495  0.0274  0.9875]
 [ 0.4353  0.1027  0.0219]
 [ 0.5178  0.3178  0.0602]
 [ 0.8767  0.4434  0.9587]
 [ 0.8628  0.7484  0.5724]
 [ 0.667   0.1453  0.3884]
 [ 0.788   0.1539  0.9882]
 [ 0.8844  0.1161  0.903 ]
 [ 0.5003  0.4942  0.5335]]
Q:
[[ 0.2732 -0.2322  0.3065  0.1374 -0.0534 -0.222   0.3219 -0.7765  0.      0.    ]
 [-0.2322  0.9258  0.0979  0.0439 -0.0171 -0.0709  0.1028 -0.2481  0.      0.    ]
 [ 0.3065  0.0979  0.8708 -0.0579  0.0225  0.0936 -0.1357  0.3274  0.      0.    ]
 [ 0.1374  0.0439 -0.0579  0.974   0.0101  0.042  -0.0608  0.1468  0.      0.    ]
 [-0.0534 -0.0171  0.0225  0.0101  0.9961 -0.0163  0.0236 -0.057   0.      0.    ]
 [-0.222  -0.0709  0.0936  0.042  -0.0163  0.9322  0.0983 -0.2371  0.      0.    ]
 [ 0.3219  0.1028 -0.1357 -0.0608  0.0236  0.0983  0.8575  0.3438  0.      0.    ]
 [-0.7765 -0.2481  0.3274  0.1468 -0.057  -0.2371  0.3438  0.1705  0.      0.    ]
 [ 0.      0.      0.      0.      0.      0.    

## Cancellation Error Test

In [83]:
# Create the bad matrix
badMat = np.random.rand(10,2)
badMat[len(badMat)-2,0] = 1*10**10
badMat[len(badMat)-1,1] = 1*10**10

In [84]:
badMat

array([[  3.88717775e-01,   8.62741507e-01],
       [  8.37233359e-01,   9.59556286e-02],
       [  2.49542282e-01,   5.69735992e-01],
       [  9.85508213e-01,   8.39130382e-01],
       [  3.36109676e-01,   8.82079434e-01],
       [  7.23717784e-01,   4.90886587e-01],
       [  3.25431160e-01,   9.33551083e-01],
       [  5.13807894e-02,   9.84516551e-01],
       [  1.00000000e+10,   2.39006829e-01],
       [  5.82627862e-01,   1.00000000e+10]])

In [85]:
#Just look at the lower matrix
householdersame(badMat)[1]

array([[ -1.00000000e+10,  -0.00000000e+00],
       [ -8.21634691e-01,  -1.00000000e+10],
       [  0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00],
       [ -0.00000000e+00,  -0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,  -0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00],
       [ -0.00000000e+00,  -0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00]])

In [86]:
#Just look at the lower matrix
householderopposite(badMat)[1]

array([[  1.00000000e+10,   2.39006829e-01],
       [  5.82627862e-01,   1.00000000e+10],
       [ -1.60930000e-06,   5.69735992e-01],
       [ -2.29480000e-06,   8.39130381e-01],
       [ -2.50340000e-06,   8.82079434e-01],
       [ -1.38580000e-06,   4.90886586e-01],
       [ -2.38420000e-06,   9.33551083e-01],
       [ -2.62260000e-06,   9.84516552e-01],
       [ -2.38420000e-06,   8.62741508e-01],
       [ -2.64500000e-07,   9.59556271e-02]])